# VibeVoice FastAPI Server on Google Colab 🚀

This notebook will clone `deepfak3voice` repository, install all necessary dependencies, and launch the FastAPI server using `uvicorn`. We'll then use `ngrok` to create a public URL so you can test the API endpoints from anywhere.

## Step 1: Setup & GPU Check

First, let's ensure you're using a GPU runtime. Go to **Runtime -> Change runtime type** and select **T4 GPU** as the hardware accelerator.

The command below will verify that the GPU is active.

In [ ]:
!nvidia-smi

## Step 2: Clone the Repository

Now, we'll clone your project from GitHub and move into the project directory.

In [ ]:
!git clone https://github.com/iamspruce/deepfak3voice.git
%cd deepfak3voice

## Step 3: Install Dependencies

We'll install all the packages listed in your `requirements.txt` file, plus `pyngrok` to manage the tunnel to our server.

In [ ]:
!pip install -q -r requirements.txt

## Step 5: Run the FastAPI Server!

This is the final step. We will start a public ngrok tunnel and then launch the uvicorn server. 

A public URL will be printed. **Click this URL** to access your API. The server logs will be displayed in this cell. To stop the server, just interrupt or stop this cell.

In [ ]:
import threading, time, re

# Start your FastAPI app in the background
def run_server():
    !uvicorn speak:app --host 0.0.0.0 --port 8000 --reload

threading.Thread(target=run_server, daemon=True).start()
time.sleep(5)  # Wait a few seconds for the server to start

# Start Cloudflare tunnel to expose the app publicly
!cloudflared tunnel --url http://localhost:8000 --no-autoupdate > cloudflare.log 2>&1 &

time.sleep(8)  # Give it a few seconds to initialize

# Extract the public URL
try:
    log_output = !grep -o 'https://[-0-9a-z]*\.trycloudflare\.com' cloudflare.log
    if log_output:
        public_url = log_output[-1]
        print(f"🚀 Your VibeVoice server is live at: {public_url}")
    else:
        print("❌ Could not extract the Cloudflare URL. Check the log below:\n")
        !tail -n 20 cloudflare.log
except Exception as e:
    print("⚠️ Error reading tunnel log:", e)